<a href="https://colab.research.google.com/github/shakti0512/2311MC13_Shakti_Singh/blob/main/Model_hyperparameters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("diabetes.csv")

In [3]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [5]:
X = df.iloc[:,:-1].values
Y = df.iloc[:,-1].values

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [7]:
X = scaler.fit_transform(X)

In [8]:
X.shape

(768, 8)

In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=1)

In [10]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense

In [11]:
model = Sequential()
model.add(Dense(32,activation = 'relu',input_dim=8))
model.add(Dense(1,activation = 'sigmoid'))

model.compile(optimizer ='Adam', loss = 'binary_crossentropy', metrics= ['accuracy'])

In [12]:
model.fit(X_train,Y_train,batch_size=32, epochs=10)

Epoch 1/10
20/20 [==============================] - 1s 2ms/step - loss: 0.6981 - accuracy: 0.5879
Epoch 2/10
20/20 [==============================] - 0s 2ms/step - loss: 0.6486 - accuracy: 0.6368
Epoch 3/10
20/20 [==============================] - 0s 3ms/step - loss: 0.6126 - accuracy: 0.6580
Epoch 4/10
20/20 [==============================] - 0s 2ms/step - loss: 0.5840 - accuracy: 0.6857
Epoch 5/10
20/20 [==============================] - 0s 2ms/step - loss: 0.5631 - accuracy: 0.6906
Epoch 6/10
20/20 [==============================] - 0s 3ms/step - loss: 0.5460 - accuracy: 0.7020
Epoch 7/10
20/20 [==============================] - 0s 2ms/step - loss: 0.5315 - accuracy: 0.7150
Epoch 8/10
20/20 [==============================] - 0s 2ms/step - loss: 0.5199 - accuracy: 0.7345
Epoch 9/10
20/20 [==============================] - 0s 3ms/step - loss: 0.5093 - accuracy: 0.7427
Epoch 10/10
20/20 [==============================] - 0s 2ms/step - loss: 0.5003 - accuracy: 0.7524


In [13]:
pip install -U keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 1.4 MB/s eta 0:00:00


In [14]:
import keras_tuner as kt

In [15]:
def build_model(hp):
  model = Sequential()
  model.add(Dense(32,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer=hp.Choice('optimizer',values=['adam','sgd','rmsprop','adadelta']),loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [16]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5)

In [17]:
tuner.search(X_train,Y_train,epochs=5,validation_data=(X_test,Y_test))

Trial 4 Complete [00h 00m 02s]
val_accuracy: 0.5649350881576538

Best val_accuracy So Far: 0.8246753215789795
Total elapsed time: 00h 00m 11s


In [18]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [19]:
model = tuner.get_best_models(num_models=1)[0]

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321 (1.25 KB)
Trainable params: 321 (1.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [21]:
model.fit(X_train,Y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(X_test,Y_test))

Epoch 7/100
20/20 [==============================] - 1s 12ms/step - loss: 0.5126 - accuracy: 0.7573 - val_loss: 0.5080 - val_accuracy: 0.8117
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5015 - accuracy: 0.7622 - val_loss: 0.5005 - val_accuracy: 0.7922
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4930 - accuracy: 0.7655 - val_loss: 0.4944 - val_accuracy: 0.8052
Epoch 10/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4851 - accuracy: 0.7687 - val_loss: 0.4900 - val_accuracy: 0.8052
Epoch 11/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4795 - accuracy: 0.7752 - val_loss: 0.4862 - val_accuracy: 0.7987
Epoch 12/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4746 - accuracy: 0.7720 - val_loss: 0.4839 - val_accuracy: 0.8117
Epoch 13/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4707 - accuracy: 0.7720 - val_loss: 0.4810 - val_accuracy: 0.8117
Epoch 14

In [36]:
def build_model(hp):
  model = Sequential()
  units = hp.Int('units',min_value=8,max_value=128,step=8)
  model.add(Dense(units=units,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
  return model


In [37]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5,
                        directory='mydir',
                        project_name='shakti')

In [38]:
tuner.search(X_train,Y_train,epochs=5,validation_data=(X_test,Y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.7922077775001526

Best val_accuracy So Far: 0.8181818127632141
Total elapsed time: 00h 00m 10s


In [39]:
tuner.get_best_hyperparameters()[0].values

{'units': 64}

In [41]:
model=tuner.get_best_models(num_models=1)[0]

In [43]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                576       
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 641 (2.50 KB)
Trainable params: 641 (2.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [44]:
model.fit(X_train,Y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(X_test,Y_test))

Epoch 7/100
20/20 [==============================] - 1s 13ms/step - loss: 0.5184 - accuracy: 0.7573 - val_loss: 0.4953 - val_accuracy: 0.8182
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5013 - accuracy: 0.7573 - val_loss: 0.4828 - val_accuracy: 0.7857
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4895 - accuracy: 0.7687 - val_loss: 0.4752 - val_accuracy: 0.7987
Epoch 10/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4803 - accuracy: 0.7752 - val_loss: 0.4674 - val_accuracy: 0.7792
Epoch 11/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4736 - accuracy: 0.7850 - val_loss: 0.4624 - val_accuracy: 0.7792
Epoch 12/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4676 - accuracy: 0.7850 - val_loss: 0.4586 - val_accuracy: 0.7857
Epoch 13/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4631 - accuracy: 0.7818 - val_loss: 0.4566 - val_accuracy: 0.7922
Epoch 14

In [45]:
def build_model(hp):
  model=Sequential()
  model.add(Dense(64,activation='relu',input_dim=8))
  for i in range (hp.Int('num_layers',min_value=1,max_value=10)):
    model.add(Dense(64,activation='relu'))
  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
  return model


In [47]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5,
                        directory='mydir',
                        project_name='shakti_layers')

In [48]:
tuner.search(X_train,Y_train,epochs=5,validation_data=(X_test,Y_test))

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.798701286315918

Best val_accuracy So Far: 0.8181818127632141
Total elapsed time: 00h 00m 14s


In [49]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 9}

In [50]:
model = tuner.get_best_models(num_models=1)[0]

In [51]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                576       
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 64)                4160      
                                                                 
 dense_5 (Dense)             (None, 64)                4160      
                                                                 
 dense_6 (Dense)             (None, 64)                4

In [52]:
model.fit(X_train,Y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(X_test,Y_test))

Epoch 7/100
20/20 [==============================] - 2s 17ms/step - loss: 0.4507 - accuracy: 0.7687 - val_loss: 0.4860 - val_accuracy: 0.7857
Epoch 8/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4395 - accuracy: 0.7785 - val_loss: 0.4811 - val_accuracy: 0.7987
Epoch 9/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4222 - accuracy: 0.8078 - val_loss: 0.4989 - val_accuracy: 0.7792
Epoch 10/100
20/20 [==============================] - 0s 9ms/step - loss: 0.4172 - accuracy: 0.8029 - val_loss: 0.4804 - val_accuracy: 0.8117
Epoch 11/100
20/20 [==============================] - 0s 9ms/step - loss: 0.4154 - accuracy: 0.7932 - val_loss: 0.4873 - val_accuracy: 0.7857
Epoch 12/100
20/20 [==============================] - 0s 8ms/step - loss: 0.3948 - accuracy: 0.8208 - val_loss: 0.5700 - val_accuracy: 0.7857
Epoch 13/100
20/20 [==============================] - 0s 8ms/step - loss: 0.3972 - accuracy: 0.8176 - val_loss: 0.4822 - val_accuracy: 0.7792
Epoch 14

In [64]:
def build_model(hp):
  model = Sequential()
  count = 0
  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
    if count == 0:
      model.add(
          Dense(
              hp.Int('units'+ str(i),min_value=8,max_value=128,step=8),
              activation= hp.Choice('activation' + str(i),values=['relu','tanh','sigmoid']),
              input_dim=8
              )
          )
    else:
      model.add(
        Dense(
            hp.Int('units'+ str(i),min_value=8,max_value=128,step=8),
            activation= hp.Choice('activation' + str(i),values=['relu','tanh','sigmoid'])
            )
        )
    count +=1
  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer=hp.Choice('optimizer',values=['rmsprop','adam','sgd','nadam','adadelta']),
                loss='binary_crossentropy',
                metrics=['accuracy'])
  return model

In [66]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5,
                        directory = 'mydir',
                        project_name='final_shakti')

In [67]:
tuner.search(X_train,Y_train,epochs=5,validation_data=(X_test,Y_test))

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.798701286315918

Best val_accuracy So Far: 0.798701286315918
Total elapsed time: 00h 00m 13s


In [68]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 4,
 'units0': 104,
 'activation0': 'sigmoid',
 'optimizer': 'adam',
 'units1': 96,
 'activation1': 'tanh',
 'units2': 64,
 'activation2': 'tanh',
 'units3': 80,
 'activation3': 'tanh',
 'units4': 112,
 'activation4': 'sigmoid',
 'units5': 24,
 'activation5': 'tanh'}

In [69]:
model = tuner.get_best_models(num_models=1)[0]

In [70]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 104)               936       
                                                                 
 dense_1 (Dense)             (None, 96)                10080     
                                                                 
 dense_2 (Dense)             (None, 64)                6208      
                                                                 
 dense_3 (Dense)             (None, 80)                5200      
                                                                 
 dense_4 (Dense)             (None, 1)                 81        
                                                                 
Total params: 22505 (87.91 KB)
Trainable params: 22505 (87.91 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [71]:
model.fit(X_train,Y_train,batch_size=32,epochs=60,initial_epoch=5,validation_data = (X_test,Y_test))

Epoch 6/60
20/20 [==============================] - 1s 15ms/step - loss: 0.4931 - accuracy: 0.7590 - val_loss: 0.4847 - val_accuracy: 0.8117
Epoch 7/60
20/20 [==============================] - 0s 5ms/step - loss: 0.4768 - accuracy: 0.7834 - val_loss: 0.5054 - val_accuracy: 0.7857
Epoch 8/60
20/20 [==============================] - 0s 5ms/step - loss: 0.4840 - accuracy: 0.7687 - val_loss: 0.5328 - val_accuracy: 0.7338
Epoch 9/60
20/20 [==============================] - 0s 6ms/step - loss: 0.5129 - accuracy: 0.7443 - val_loss: 0.4887 - val_accuracy: 0.8117
Epoch 10/60
20/20 [==============================] - 0s 5ms/step - loss: 0.4824 - accuracy: 0.7622 - val_loss: 0.4822 - val_accuracy: 0.7662
Epoch 11/60
20/20 [==============================] - 0s 4ms/step - loss: 0.5069 - accuracy: 0.7638 - val_loss: 0.4861 - val_accuracy: 0.7727
Epoch 12/60
20/20 [==============================] - 0s 5ms/step - loss: 0.4787 - accuracy: 0.7720 - val_loss: 0.4688 - val_accuracy: 0.7727
Epoch 13/60
20/2